In [24]:
from pathlib import Path
import pandas as pd 
import arrow

In [25]:
dataset_path = Path().absolute()

In [26]:
def read_df(in_filepath: Path) -> pd.DataFrame:

    return pd.read_csv(in_filepath)

In [27]:
flight_path = dataset_path / "flight_data" / "Enterprise-II-Data" / "concatenated_data"
weather_path = dataset_path / "weather-data"
output_path = dataset_path / "final-weather-flight-data"

In [28]:
flights = read_df(flight_path/"Full_Departures_Data.csv")
weather = read_df(weather_path / "Enterprise-II-YUL-Full-Weather.csv")

In [29]:
flights.head(2)

,Flight Status,Departure Airport,Departure Gate,Arrival IATA Code,Scheduled Departure,Actual Departure,Delay (in Minutes),Airline Name,Flight Number
0,landed,YUL,A63,DXB,2024-04-08T10:20:00+00:00,2024-04-08T10:28:00+00:00,8.0,Emirates,EK244
1,landed,YUL,C82,EWR,2024-04-08T08:15:00+00:00,2024-04-08T08:43:00+00:00,28.0,Air Canada,AC8938


In [30]:
# can I write a function to do this? Yes. 
# writing it out to test if a vectorized operation is better than writing a function

flights['Scheduled Departure'] = flights['Scheduled Departure'].apply(
    lambda x: arrow.get(x).format('YYYY-MM-DD HH:00:00') if isinstance(x, str) else x)

flights['Actual Departure'] = flights['Actual Departure'].apply(
    lambda x: arrow.get(x).format('YYYY-MM-DD HH:00:00') if isinstance(x, str) else x)


In [32]:
flights['Scheduled Departure'] = flights['Scheduled Departure'].apply(
    lambda x: arrow.get(x).format("YYYY-MM-DD HH:00:00") if pd.notna(x) else x)

flights['Actual Departure'] = flights['Actual Departure'].apply(
    lambda x: arrow.get(x).format("YYYY-MM-DD HH:00:00") if pd.notna(x) else x)

In [34]:
flights_and_weather = flights.merge(weather, how = "left", left_on = "Scheduled Departure", right_on = "Timestamp")

In [36]:
flights_and_weather.to_csv(output_path / "Enterprise-II-YUL-Flights-Weather.csv", index = False)